In [127]:
import numpy as np 
import os
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import KFold
from collections import defaultdict
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from pprint import pprint
from sklearn.model_selection import RandomizedSearchCV
import statistics
from sklearn.metrics import make_scorer, accuracy_score, roc_auc_score 
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from collections import Counter

In [128]:
filenames = os.listdir('./searched_songs/')
filepath = "./searched_songs/"

In [129]:
features = ["acousticness", "danceability", "energy", "instrumentalness", "key", "liveness",
                "mode", "loudness", "speechiness", "tempo", "valence","length","time_signature"]

In [130]:
threshold=0

In [131]:
sum_rfaccuracy=0
count=0
lowest_accuracy = 100
whole_dataset = pd.read_csv(filepath+"us.csv", encoding='utf-8',)

In [132]:
whole_dataset.head()
# whole_dataset = whole_dataset.drop("Unnamed:0")


,Unnamed: 0,name,album_name,artist_name,artist_id,album_release_date,album_type,available_markets,length,popularity,...,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,valence,tempo,time_signature
0,0,Guarda,Zéphyrine,Elliot Ziegler,0Wh30jSV0ZQf0IzHkEddBY,2019-05-25,single,"['AD', 'AE', 'AL', 'AR', 'AT', 'AU', 'BA', 'BE...",98750,59,...,0.99400,0.334,0.0471,0.96600,0.1070,-23.170,0.0496,0.313,136.455,4
1,1,Habit,Xxotic,Why Cue,5PLUeP0AXBzIb4yrN7rQh6,2019-01-11,album,"['AD', 'AE', 'AL', 'AR', 'AT', 'AU', 'BA', 'BE...",161045,45,...,0.06680,0.803,0.8190,0.00177,0.0889,-7.094,0.0504,0.344,125.012,4
2,2,BLOODY STREAM,BLOODY STREAM,Coda,2xcKG6XtsW4NV5zMXWuL6n,2013-01-30,single,"['AD', 'AE', 'AR', 'AT', 'AU', 'BE', 'BG', 'BH...",261920,69,...,0.00860,0.274,0.9030,0.00000,0.3150,-3.125,0.0640,0.662,129.999,4
3,3,The Voice,The Voice,Lil Durk,3hcs9uc56yIGFCSy9leWe7,2020-09-04,single,"['AD', 'AE', 'AL', 'AR', 'AT', 'AU', 'BA', 'BE...",179724,69,...,0.02590,0.534,0.8190,0.00000,0.1240,-2.099,0.1020,0.129,116.632,5
4,4,Belle Belle,Belle Belle,Fran Zcordion,2Vvs6Uyt1O7vpcT7ljgfL6,2020-06-30,single,"['AD', 'AE', 'AL', 'AR', 'AT', 'AU', 'BA', 'BE...",137848,47,...,0.00755,0.624,0.6680,0.56000,0.1860,-10.245,0.0325,0.717,139.992,4


In [133]:
# df.loc[:, ~df.columns.str.match('Unnamed')]
whole_dataset = whole_dataset.loc[:, ~whole_dataset.columns.str.match('Unnamed')]

In [134]:
whole_dataset.head()
print(len(whole_dataset))

18753


In [135]:
def detect_outliers(df, features):
    outlier_indices = []

    for c in features:
        # 1st quartile
        Q1 = np.percentile(df[c], 25)
        # 3rd quartile
        Q3 = np.percentile(df[c], 75)
        # IQR
        IQR = Q3 - Q1
        # Outlier step
        outlier_step = IQR * 1.5
        # detect outlier and their indeces
        outlier_list_col = df[(df[c] < Q1 - outlier_step)
                              | (df[c] > Q3 + outlier_step)].index  # filtre
        # store indeces
        # The extend() extends the list by adding all items of a list (passed as an argument) to the end.
        outlier_indices.extend(outlier_list_col)

    outlier_indices = Counter(outlier_indices)
    multiple_outliers = list(i for i, v in outlier_indices.items() if v > 2)

    return multiple_outliers



In [136]:
outlier_features = ["popularity","length","danceability","energy","instrumentalness","liveness","loudness","speechiness","valence"]
whole_dataset.loc[detect_outliers(whole_dataset,outlier_features)]
print(len(whole_dataset.loc[detect_outliers(whole_dataset,outlier_features)]))

1943


In [137]:
# whole_dataset = whole_dataset.drop(detect_outliers(whole_dataset, outlier_features),axis = 0).reset_index(drop = True)

In [138]:
print(len(whole_dataset))

18753


In [139]:
# threshold = whole_dataset["popularity"].quantile(0.70)

# df[df.a < df.a.quantile(.95)]

# threshold_df = whole_dataset.nlargest(int(len(whole_dataset.index)*0.60),'popularity')


# print(threshold_df.head())

# print(len(threshold_df))

# threshold = np.mean(threshold_df.popularity)
# print(f"type of threshold {type(threshold)}")
whole_dataset["thresholded_popularity"]= [ 1 if i>=50 else 0 for i in whole_dataset.popularity ]
whole_dataset["thresholded_popularity"].value_counts()
# threshold=80
# print(f"threshold {threshold}")
# whole_dataset.loc[whole_dataset['popularity'] < threshold, 'thresholded_popularity'] = 0
# whole_dataset.loc[whole_dataset['popularity']
#                   >= threshold, 'thresholded_popularity'] = 1
print(whole_dataset.thresholded_popularity.value_counts())

1    14518
0     4235
Name: thresholded_popularity, dtype: int64


In [140]:
def change_type(var):
    whole_dataset[var] = whole_dataset[var].astype(int)

In [141]:
column= ["key","length","mode"]
for i in column:
    change_type(i)



In [142]:
x,y = whole_dataset[features] , whole_dataset.loc[:,'thresholded_popularity']
# x = (x - np.min(x))/(np.max(x)-np.min(x)).values
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state=42)
y=y.astype(int)
y_train= y_train.astype(int)
y_test= y_test.astype(int)
x_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15002 entries, 2559 to 15795
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   acousticness      15002 non-null  float64
 1   danceability      15002 non-null  float64
 2   energy            15002 non-null  float64
 3   instrumentalness  15002 non-null  float64
 4   key               15002 non-null  float64
 5   liveness          15002 non-null  float64
 6   mode              15002 non-null  float64
 7   loudness          15002 non-null  float64
 8   speechiness       15002 non-null  float64
 9   tempo             15002 non-null  float64
 10  valence           15002 non-null  float64
 11  length            15002 non-null  float64
 12  time_signature    15002 non-null  float64
dtypes: float64(13)
memory usage: 1.6 MB


In [143]:
whole_dataset.columns[whole_dataset.isnull().any()]

# song_data.columns[song_data.isnull().any()]
whole_dataset.isnull().sum()



name                      0
album_name                0
artist_name               0
artist_id                 0
album_release_date        0
album_type                0
available_markets         0
length                    0
popularity                0
trackid                   0
mode                      0
key                       0
acousticness              0
danceability              0
energy                    0
instrumentalness          0
liveness                  0
loudness                  0
speechiness               0
valence                   0
tempo                     0
time_signature            0
thresholded_popularity    0
dtype: int64

In [144]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier(n_estimators=150,random_state = 3)
rf.fit(x_train,y_train)
print("Train ccuracy of random forest",rf.score(x_train,y_train))
print("Test accuracy of random forest",rf.score(x_test,y_test))
RandomForestClassifier_score=rf.score(x_test,y_test)
y_pred=rf.predict(x_test)
t_true=y_test

Train ccuracy of random forest 0.9964671377149713
Test accuracy of random forest 0.8944281524926686


In [145]:
cm = confusion_matrix(y_test, y_pred)
print('Confusion matrix: \n', cm)

Confusion matrix: 
 [[ 516  317]
 [  79 2839]]


In [146]:
cr = classification_report(y_test, y_pred,output_dict=False)
print('Classification report: \n',cr
        )

Classification report: 
               precision    recall  f1-score   support

           0       0.87      0.62      0.72       833
           1       0.90      0.97      0.93      2918

    accuracy                           0.89      3751
   macro avg       0.88      0.80      0.83      3751
weighted avg       0.89      0.89      0.89      3751



In [147]:
# print(type(cr))
del(cr["accuracy"])
sum = 0
i=0
for key,value in cr.items():
    if i==2:
        break
    sum+=value["precision"]
    i+=1
# print(sum)

TypeError: 'str' object does not support item deletion